In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_buildstock_golive
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
/* 
SELECT DISTINCT T2.BUSINESS_UNIT, '1345' DC_ID, T1.PRODUCT_ID, '1344' DC_BS_ID
FROM `VCM_DWH_PRD.D_ASSORTMENT_HERO` T1
INNER JOIN (SELECT DISTINCT BUSINESS_UNIT, CITY FROM VCM_DWH_PRD.D_PORTAL_BU_REGION_DC_CITY WHERE REGION = 'DC CẦN THƠ' AND BUSINESS_UNIT = '2000') T2
USING (BUSINESS_UNIT, CITY)
*/
SELECT DISTINCT
    business_unit, dc_id, product_id, dc_bs_id
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_portal_winare_buildstock_golive;
""")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_buildstock_golive_his
(
  calday DATE,
  business_unit STRING,
  dc_id STRING,
  product_id STRING,
  dc_bs_id STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_buildstock_golive_his WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS);
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_buildstock_golive_his
SELECT 
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) calday,
    business_unit, 
    dc_id, 
    product_id, 
    dc_bs_id
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_buildstock_golive;
""")